In [100]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [101]:
import os
from typing import List
import sys
import uuid

sys.path.append("../../llm_app_eval")
from qa_extraction import QuestionAnswerPairs, extract_question_answer_pairs, save_qa_pairs
from llm_app import BaseApp, InputFormat, OutputFormat
from evaluator import TestCase, EvalProperty, PropertyResult, Evaluator

import openai
import instructor

openai.api_key_path = "../../../openai_key"
instructor.patch()

In [51]:
# Load documents
def load_documents(path):
    docs = []
    for filename in os.listdir(path):
        with open(path + filename) as f:
            docs.append(f.read())
    return docs

def chunk_markdown(md_doc: str) -> List[str]:
    """Split markdown document into chunks"""
    chunks = []
    chunk = ""
    for line in md_doc.splitlines():
        if line.startswith("# "):
            chunks.append(chunk)
            chunk = ""
        chunk += line + "\n"
    chunks.append(chunk)
    return chunks[1:]

In [52]:
# Load documents
docs = load_documents("../data/documents/")
print(f"Loaded {len(docs)} documents")
print(docs)

# Chunk documents
chunks = []
for doc in docs:
    chunks.extend(chunk_markdown(doc))

Loaded 1 documents
['# 2021 - EHBO\n\n# Vier stappen van eerste hulp\n\n1. Zorg voor veiligheid, van jezelf, het slachtoffer (SO) en de omstaanders.\n2. Beoordeel de toestand van het SO.\n    1. Bewustzijn: verbaal + schudden met schouders\n        1. Bewust? → laten liggen, vragen wat er gebeurd is\n        2. Bewusteloos? → Controleer ademhaling: open de luchtweg met kinlift + 10 tellen ademhaling observeren (kijken naar borst, horen en voelen aan mond)\n            1. Ademhaling → 112 bellen→ stabiele zijligging + ademhaling blijven controleren\n            2. Geen ademhaling → 112 bellen → reanimeren\n3. Hulpdiensten verwittigen indien nodig:\n    1. 112, 101, antigifcentrum (070 245 245), huisartsenwachtpost (1733)\n    2. 112: vermeld waar, wat en wie\n4. Verleen verdere eerste hulp\n\n# Algemeen\n\n- Vermijd het zeggen van "het komt allemaal goed". Maak **geen valse beloftes**. Zeg wel: "de hulpdiensten zijn verwittigd en komen eraan", "ik blijf bij u tot de hulpdiensten er zijn

In [38]:
# Extract question answer pairs
question_answer_pairs = []
for chunk in chunks:
    qas = extract_question_answer_pairs(chunk)
    question_answer_pairs.extend(qas)

In [46]:
save_qa_pairs(question_answer_pairs, "../data/question_answer_pairs.csv")